In [240]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import string
import datetime as dt
from math import *

import yfinance as yf
import sp_components_data as sp
# from fmp_python import company_valuation

import sqlite3


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
pricing_date = '2024-08-30' 
total_num_stocks = 30

## Getting the data

### S&P constituent price data

#### 12-1 momentum  (total return)
The 12-1 measures the performance of any stock over the past 12 months, excluding the most recent month. The month recent month is excluded because short-term price movements tend to reverse, so removing it produces a stronger momentum signal.

#### Method 1

Momentum from return data

In [7]:
# tickers = sp.get_sp_tickers()
# data = yf.download(tickers, period="13mo", interval="1d")
# index_data = yf.download(index_ticker, period="13mo", interval="1d")
# price data
# sp500 = yf.Ticker("^GSPC")
# sp500_constituents = sp500.history(period="1d")
# print(sp500_constituents)
with open('sp_components_data.pkl', 'rb') as f:
    data = pickle.load(f)
price_data = data['Adj Close']
data.head(3)

Price                      Adj Close                                 \
Ticker                             A        AAL      AAPL ABBV ABNB   
Date                                                                  
2008-01-02 00:00:00+00:00  23.293402  12.557317  5.876340  NaN  NaN   
2008-01-03 00:00:00+00:00  23.062395  11.991672  5.879057  NaN  NaN   
2008-01-04 00:00:00+00:00  22.311621  11.699421  5.430277  NaN  NaN   

Price                                                                 \
Ticker                           ABT      ACGL        ACN       ADBE   
Date                                                                   
2008-01-02 00:00:00+00:00  18.215652  7.608889  26.544682  41.709999   
2008-01-03 00:00:00+00:00  18.104681  7.764444  26.088289  41.790001   
2008-01-04 00:00:00+00:00  18.209129  7.702222  25.698139  40.360001   

Price                                 ...  Volume                             \
Ticker                           ADI  ...     WTW       WY     WYNN      XEL   
Date                                  ...                                      
2008-01-02 00:00:00+00:00  20.164034  ...  329746  1867500  1629800  1752000   
2008-01-03 00:00:00+00:00  19.858608  ...  320422  2158200  2432900  1589400   
2008-01-04 00:00:00+00:00  19.340736  ...  478406  2988500  3068700  2448400   

Price                                                                  
Ticker                          XOM XYL      YUM      ZBH    ZBRA ZTS  
Date                                                                   
2008-01-02 00:00:00+00:00  23351900 NaN  4132244  1476608  720200 NaN  
2008-01-03 00:00:00+00:00  19160600 NaN  3846950  1570441  467600 NaN  
2008-01-04 00:00:00+00:00  24730900 NaN  6056692  2314101  401400 NaN  

[3 rows x 3018 columns]

Get daily returns from prices data.

In [15]:
price_data_13mo = price_data["2023-08-01":]
daily_returns = price_data_13mo.pct_change()
daily_returns.head(3)

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-02 00:00:00+00:00,0.042701,-0.017252,-0.015490,0.005655,-0.029212,-0.017280,0.003372,-0.009939,-0.034238,-0.035191,...,0.002634,-0.022667,-0.036467,0.000477,-0.012474,0.001340,-0.009974,-0.029410,-0.053460,-0.018921
2023-08-03 00:00:00+00:00,-0.020828,0.001881,-0.007322,-0.002209,-0.025457,-0.009943,0.000388,-0.002045,-0.012333,-0.005040,...,0.002956,-0.002711,0.004279,-0.024809,0.017381,-0.032834,0.004593,-0.020174,0.034833,-0.006154


In [41]:
def calc_12_1_momentum_1(daily_returns, months = 12, exclude_month = 1):
    # convert daily returns to monthly cumulative total returns
    monthly_returns = daily_returns.resample('M').agg(lambda x: (x + 1).prod())
    # to get monthly returns: df.resample('M').agg(lambda x: (x + 1).prod() - 1)
    momentum = monthly_returns[-months-exclude_month:-exclude_month].prod()
    return momentum

In [50]:
momentum = calc_12_1_momentum(daily_returns)
momentum_df = pd.DataFrame(momentum, columns=["Momentum"]).reset_index()
momentum_df.head()

,Ticker,Momentum
0,A,1.162795
1,AAL,0.655576
2,AAPL,1.141350
3,ABBV,1.295813
4,ABNB,0.937210


#### Method 2

Use price data but also take into account dividend earned

Let $R231_{i,t}$ be defined as the 11-month total return computed from 231 business day total returns for asset $i$ at time $t$. It is calculated as continuously compounded total return for the price and dividends over the prior 11-month period. 

$PRCCD_{231,i}$ is the price close daily for asset $i$  
$DivPerShare_i$ is dividend per share for asset $i$

$R231_{i,t} = \frac{PRCCD_{231,i}}{PRCCD_{0,i}}\prod_{n=0}^{231}{(1+\frac{DivPerShare_{n,i}}{PRCCD_{n,i}})}-1$

In [124]:
with open('sp_div_ts_data.pkl', 'rb') as f:
    div_data = pickle.load(f)
div_data.head(3)

MMM             AOS             ABT  \
                               Close Dividends Close Dividends Close   
Date                                                                   
2022-09-14 00:00:00-04:00  88.996834       0.0   NaN       NaN   NaN   
2022-09-15 00:00:00-04:00  88.156303       0.0   NaN       NaN   NaN   
2022-09-16 00:00:00-04:00  88.300186       0.0   NaN       NaN   NaN   

                                     ABBV             ACN            ...  \
                          Dividends Close Dividends Close Dividends  ...   
Date                                                                 ...   
2022-09-14 00:00:00-04:00       NaN   NaN       NaN   NaN       NaN  ...   
2022-09-15 00:00:00-04:00       NaN   NaN       NaN   NaN       NaN  ...   
2022-09-16 00:00:00-04:00       NaN   NaN       NaN   NaN       NaN  ...   

                            XYL             YUM            ZBRA            \
                          Close Dividends Close Dividends Close Dividends   
Date                                                                        
2022-09-14 00:00:00-04:00   NaN       NaN   NaN       NaN   NaN       NaN   
2022-09-15 00:00:00-04:00   NaN       NaN   NaN       NaN   NaN       NaN   
2022-09-16 00:00:00-04:00   NaN       NaN   NaN       NaN   NaN       NaN   

                            ZBH             ZTS            
                          Close Dividends Close Dividends  
Date                                                       
2022-09-14 00:00:00-04:00   NaN       NaN   NaN       NaN  
2022-09-15 00:00:00-04:00   NaN       NaN   NaN       NaN  
2022-09-16 00:00:00-04:00   NaN       NaN   NaN       NaN  

[3 rows x 998 columns]

In [173]:
div_data_13mo = div_data["2023-08-01":]
valid_tickers = div_data_13mo.columns.get_level_values(0).unique().tolist()
start_date = (pd.to_datetime(pricing_date) + pd.DateOffset(days=-231)).strftime('%Y-%m-%d')
moms = []
for t in valid_tickers:
    sub_df = div_data_13mo[t][start_date:pricing_date].dropna()
    sub_df['d/p'] = sub_df.Dividends / sub_df.Close
    moms.append((sub_df['d/p']+1).prod() + sub_df.Close[-1] / sub_df.Close[0] + 1)
mom_df = pd.DataFrame.from_dict({'Ticker':valid_tickers, 'R231':moms})
mom_df

,Ticker,R231
0,MMM,3.562573
1,AOS,3.055813
2,ABT,3.014963
3,ABBV,3.260162
4,ACN,2.983167
...,...,...
494,XYL,3.249203
495,YUM,3.075492
496,ZBRA,3.387793
497,ZBH,2.946056


### Company Rating

In [188]:
rating_data = pd.read_csv('corporate_rating.csv')
rating_data['Date'] = pd.to_datetime(rating_data['Date'])
date_df = rating_data.groupby(['Symbol']).agg(rating_date = ('Date', np.max))
rating_data = rating_data.merge(date_df, left_on='Symbol', right_on='Symbol')
rating_data = rating_data[rating_data['Date'] == rating_data.rating_date].reset_index()
rating_data.head(3)

,index,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,...,freeCashFlowOperatingCashFlowRatio,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,rating_date
0,4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,2016-10-24,Consumer Durables,0.957844,0.495432,0.141608,...,0.451372,7.135348,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790,2016-10-24
1,7,A,Schlumberger N.V.,SLB,Egan-Jones Ratings Company,2015-07-23,Energy,1.905814,1.544791,0.197791,...,0.726292,5.047356,10.287293,1.908484,0.081212,15.264562,6.949487,0.248203,3.665200,2015-07-23
2,9,A,Honeywell International Inc.,HON,Moody's Investors Service,2016-02-16,Capital Goods,1.091557,0.850961,0.296935,...,0.803264,5.618107,9.692229,2.697369,0.170706,10.362903,6.994101,0.141365,4.793369,2016-02-16


#### S&P stocks ticker data

In [190]:
# ticker_data = sp.sp_ticker_data()
with open('sp_ticker_data.pkl', 'rb') as f:
    ticker = pickle.load(f)
ticker.head(3)

,symbol,sector,currentPrice,ebitdaMargins,profitMargins,grossMargins,revenueGrowth,operatingMargins,recommendationKey,earningsGrowth,...,trailingAnnualDividendRate,averageVolume10days,dividendRate,trailingPE,regularMarketVolume,averageVolume,volume,fiveYearAvgDividendYield,dividendYield,trailingPegRatio
0,MMM,Industrials,134.69,0.22536,0.02891,0.44763,-0.004,0.08169,buy,NaN,...,5.21,3277340,2.80,52.408560,6091739,4286588,6091739,4.36,0.0208,NaN
1,AOS,Industrials,83.72,0.21099,0.14674,0.38351,0.066,0.20238,hold,0.019,...,1.26,640560,1.28,21.633076,1007063,916825,1007063,1.74,0.0153,2.0570
2,ABT,Healthcare,113.27,0.25957,0.13645,0.55475,0.040,0.17664,buy,-0.051,...,2.16,3905220,2.20,35.731860,4320046,6347933,4320046,1.63,0.0194,4.5813


In [191]:
full_df = ticker.merge(rating_data, left_on='symbol', right_on='Symbol', how = 'left' )

In [192]:
no_rating = full_df[full_df.Rating.isnull() == True]
pd.unique(full_df.Rating)

array(['AA', nan, 'BBB', 'A', 'BB', 'B', 'C', 'AAA'], dtype=object)

In [193]:
full_df = full_df.merge(mom_df, left_on='symbol', right_on='Ticker', how='left')
full_df.head(3)

,symbol,sector,currentPrice,ebitdaMargins,profitMargins,grossMargins,revenueGrowth,operatingMargins,recommendationKey,earningsGrowth,...,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,rating_date,Ticker,R231
0,MMM,Industrials,134.69,0.22536,0.02891,0.44763,-0.004,0.08169,buy,NaN,...,3.06266,2.794499,0.225375,12.345138,10.262148,0.212063,9.080874,2015-10-27,MMM,3.562573
1,AOS,Industrials,83.72,0.21099,0.14674,0.38351,0.066,0.20238,hold,0.019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,AOS,3.055813
2,ABT,Healthcare,113.27,0.25957,0.13645,0.55475,0.040,0.17664,buy,-0.051,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,ABT,3.014963


### Construction of portfolio

$MinRank30_s = max(1, \lfloor{SecWt_s * 30}\rfloor)$, min number of stocks in each sector.  
Remainder stocks: $Unallocated30_s = 30 - \sum_{s}MinRank30_s$  
$SecRank30(s)$ is a ranking function where the highest value of $SecWt_s * 30$ is 1.  
We have $MaxRank30_s = \begin{cases} 
                  &MinRank30_s + 1, & SecRank30(s) \le Unallocated30_s \\                  
                  &MinRank30_s, & SecRank30(s) > Unallocated30_s
                \end{cases}$ 
to prefer sector with greater weights.   
$MaxRank30_s$ is the target allocation of stocks. 

In [289]:
sector_weights = pd.read_csv('sector_weights.csv', usecols = [1,2])
sector_weights['num_stocks'] = sector_weights.Weight * total_num_stocks
sector_weights['minrank'] = sector_weights.apply(lambda x:max(1, int(x['num_stocks'])), axis=1)
sector_weights = sector_weights.sort_values(by='num_stocks', ascending=False).reset_index()
sector_weights['secrank'] = sector_weights.index + 1
unallocated = total_num_stocks - sector_weights.minrank.sum()
sector_weights['maxrank'] = sector_weights.minrank + (sector_weights.secrank <= unallocated)
sector_weights

,index,Sector,Weight,num_stocks,minrank,secrank,maxrank
0,2,Technology,0.300976,9.029286,9,1,10
1,8,Communication Services,0.129858,3.895727,3,2,4
2,4,Financial Services,0.127863,3.835889,3,3,4
3,1,Healthcare,0.113935,3.418046,3,4,3
4,6,Consumer Cyclical,0.097383,2.921491,2,5,2
5,0,Industrials,0.071126,2.133787,2,6,2
6,9,Consumer Defensive,0.062324,1.869720,1,7,1
7,10,Energy,0.032426,0.972785,1,8,1
8,3,Utilities,0.023360,0.700792,1,9,1
9,7,Real Estate,0.022239,0.667179,1,10,1


In [290]:
sector_counts = full_df.groupby('sector').agg({'symbol':pd.Series.nunique}).reset_index()
high_ratings = ['AA', 'BBB', 'A', 'AAA']
relaxed_ratings = ['BB', 'B', 'C']
selected_stocks = []
for sector, stocks_required in zip(sector_weights.Sector, sector_weights.maxrank):
    sector_stocks = full_df[(full_df.sector == sector) & full_df.Rating.isin(high_ratings)]
    relaxed_i = 0
    while(stocks_required < len(sector_stocks)) and relaxed_i < len(relaxed_ratings):
        # relax the quality requirement
        sector_stocks = full_df[(full_df.sector == sector) & \
                                full_df.Rating.isin(high_ratings+[relaxed_ratings[relaxed_i]])]
        relaxed_i += 1
    
    sector_stocks = sector_stocks.sort_values(by=['R231'],ascending = False)
    selected_df = sector_stocks.head(stocks_required)
    selected_stocks.append(selected_df)
selected_df = pd.concat(selected_stocks)    

### Backtesting

Test our strategy using historical data, i.e. from 12/31/2000 to 12/31/2023, and assess metrics like drawdown and momentum score (12-month total return excluding the 1-month total return) of the portfolio. 